In [ ]:
import sys
sys.path.append("../src")

from preprocessing import *
from feature_extraction import *

import pandas as pd
import numpy as np
import logging

In [ ]:
volunteer_ids = [f"S{n:03d}" for n in range(1,110)]
channel_ids = ['FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'C5', 
               'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'CP5', 'CP3', 'CP1', 
               'CPz', 'CP2', 'CP4', 'CP6', 'Fp1', 'Fpz', 'Fp2', 'AF7', 
               'AF3', 'AFz', 'AF4', 'AF8', 'F7', 'F5', 'F3', 'F1', 'Fz', 
               'F2', 'F4', 'F6', 'F8', 'FT7', 'FT8', 'T7', 'T8', 'T9', 
               'T10', 'TP7', 'TP8', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 
               'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'O1', 
               'Oz', 'O2', 'Iz']
occipital_channels = ['PO7', 'PO3', 'POz', 'PO4', 'PO8', 'O1', 'Oz', 'O2']
root_dir = '../data/files/'
result_dir = '../results/tmp/'


CASE OF UNFILTERED DATA

In [ ]:
results = []
for v in volunteer_ids:  
    r1, r2 = extract_data(v, root_dir)
    
    labels = r1.ch_names
    label_idx = {label: idx for idx, label in enumerate(labels)}
 
    r1_data = r1.get_data()
    r2_data = r2.get_data()

    for ch in channel_ids:
        idx = label_idx[ch]
        signal1 = r1_data[idx,:]
        signal2 = r2_data[idx,:]
        
        etc_1 = etc_func(signal1)
        etc_2 = etc_func(signal2)

        results.append({
            'volunteer': v,
            'channel': ch,
            'ETC_EyesOpen': etc_1,
            'ETC_EyesClosed': etc_2,
            })
data_unfiltered = pd.DataFrame(results)
%store data_unfiltered


CASE OF FILTERED DATA


ETC W/O ORDINAL ANALYSIS

In [ ]:
results = []
for v in volunteer_ids:   
    r1, r2 = extract_data(v, root_dir)
    r1, r2 = filter_data(r1, r2)

    labels = r1.ch_names
    label_idx = {label: idx for idx, label in enumerate(labels)}

    r1_data = r1.get_data()
    r2_data = r2.get_data()

    for ch in channel_ids:
        idx = label_idx[ch]
        signal1 = r1_data[idx,:]
        signal2 = r2_data[idx,:]
        
        etc_1 = etc_func(signal1)
        etc_2 = etc_func(signal2)

        results.append({
            'volunteer': v,
            'channel': ch,
            'ETC_EyesOpen': etc_1,
            'ETC_EyesClosed': etc_2,
            })
data_filtered = pd.DataFrame(results)
%store data_filtered


ETC WITH ORDINAL ANALYSIS

In [ ]:
results = []
logger.info(f"Processing {len(volunteer_ids)} volunteers with {len(channel_ids)} channels each")
    
for i, v in enumerate(volunteer_ids, 1):
    logger.info(f"Processing {v} ({i}/{len(volunteer_ids)})")
    
    r1, r2 = extract_data(v, root_dir)
    r1, r2 = filter_data(r1, r2)
    r1_data = r1.get_data()
    r2_data = r2.get_data()

    channel_idx_map = {name: idx for idx, name in enumerate(channel_names)}

    for ch in channel_ids:   
        try:
            idx = channel_idx_map[ch]
            signal1 = data1[idx, :]
            signal2 = data2[idx, :]
            
            
            pattern_to_symbol, n = permute_patterns(TIME_WINDOW)
            
            # Compute ordinal patterns
            symbols1 = ordinal_patterns(pattern_to_symbol, signal1, TIME_WINDOW)
            symbols2 = ordinal_patterns(pattern_to_symbol, signal2, TIME_WINDOW)
            
            # Compute ETC
            etc1 = compute_etc(symbols1, bins=n)
            etc2 = compute_etc(symbols2, bins=n)
            
            results.append({
                'volunteer': volunteer_id,
                'channel': channel,
                'ETC_EyesOpen': etc1,
                'ETC_EyesClosed': etc2,
            })
            
        except Exception as e:
            logger.error(f"Error processing channel {channel} for {volunteer_id}: {e}")
            continue
        # Progress update
        if i % 10 == 0:
            logger.info(f"Completed {i}/{len(volunteer_ids)} volunteers")
    
    logger.info(f"Processing complete. Total results: {len(results)}")

data_filtered_ordinal = pd.DataFrame(results)
%store data_filtered_ordinal